In [ ]:
import pandas as pd
dfA = pd.read_csv('./data/Answer.csv')
dfA2 = dfA.sort_values(by=['UserID', 'QuestionID'], ascending=True)
dfA2

In [ ]:
import pandas as pd
dfQ = pd.read_csv('./data/Question.csv')
dfQ.head(20)

In [ ]:
import pandas as pd
dfS = pd.read_csv('./data/Survey.csv')
dfS

In [30]:
dfA2.loc[dfA2['QuestionID'] == 1, 'AnswerText'] = dfA2.AnswerText.str.replace('-','')
dfA2

,AnswerText,SurveyID,UserID,QuestionID
0,37,2014,1,1
1260,Female,2014,1,2
2520,United States,2014,1,3
3780,Illinois,2014,1,4
5040,-1,2014,1,5
...,...,...,...,...
228932,3,2019,4218,85
229284,reduce stigma. offer options for part time wor...,2019,4218,86
229636,I've had to drive all of the progress in menta...,2019,4218,87
229988,1,2019,4218,88


In [32]:
dfA2.loc[dfA2['UserID'] == 1]

,AnswerText,SurveyID,UserID,QuestionID
0,37,2014,1,1
1260,Female,2014,1,2
2520,United States,2014,1,3
3780,Illinois,2014,1,4
5040,-1,2014,1,5
6300,No,2014,1,6
7560,1,2014,1,7
8820,6-25,2014,1,8
10080,1,2014,1,9
11340,Yes,2014,1,10
